In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from pymongo import MongoClient
from tqdm import tqdm

In [7]:
mongo_connection = ""

In [34]:
MULT_COUNTY_JAIL_BOOKINGS_URL = "http://www.mcso.us/PAID/Home/SearchResults"

In [9]:
client = MongoClient(mongo_connection)

In [10]:
db = client.data

In [11]:
db.charges.drop()

In [35]:
response = requests.post(MULT_COUNTY_JAIL_BOOKINGS_URL)

In [36]:
soup = BeautifulSoup(response.text, 'html.parser')

In [37]:
table = soup.find('table')
table_input = table.prettify()

In [38]:
# parse dates of column at index 1
table_to_df = pd.read_html(table_input, parse_dates=[1])[0]

# sort by date descending
table_to_df = table_to_df.sort_values(by=["Booking Date", "Name"], ascending = [False, True])

In [39]:
links = table.find_all('a')

In [40]:
charge_items = list()
for link in tqdm(links):
    booking_url = link.get('href')
    inmate_link = "http://www.mcso.us" + booking_url
    
    # go to inmate's page
    inmate_response = requests.get(inmate_link)
    soup = BeautifulSoup(inmate_response.text, 'html.parser')
    
    # get table and just grab name
    table_input = soup.find('table').prettify()
    booking_table = pd.read_html(table_input)[0][1]
    
    
    charge_infos = soup.find(id='charge-info')
    charges = charge_infos.select('div > div > ol')
    for charge in charges:
        for el in charge.find_all('li'):
            charge_type = el.find('span').getText()
            charge_bail = el.find(class_='charge-bail-display').text
            charge_status = el.find(class_='charge-status-value').text
            charge_item = {
                'Type': charge_type,
                'Bail': charge_bail,
                'Status': charge_status,
                'Name': booking_table[1]
            }
            charge_items.append(charge_item)
#             db.charges.insert_one(charge_item)
            
    

100%|███████████████████████████████████████████████████████████████████████████| 882/882 [03:43<00:00,  3.95it/s]


In [41]:
len(charge_items)

9407

In [34]:
db.inmates_charges.count_documents({})

9571

In [32]:
docs = db.inmates_charges.find({})

In [35]:
charges_df = pd.DataFrame(charge_items)

In [37]:
charges_df.to_csv("csvs/inmates_charges.csv", index=False)

In [30]:
inmates_charges = [doc for doc in docs]

In [31]:
inmates_charges_df = pd.DataFrame(inmates_charges)

In [2]:
inmates_details_df = pd.read_csv("csvs/inmate_details.csv")

In [3]:
import datetime 

currentDate = datetime.date.today().strftime("%Y-%m-%d")

print(currentDate)

2022-10-04


In [4]:
inmates_details_df

,SWIS ID,Name,Age,Gender,Race,Height,Weight,Hair,Eyes,Arresting Agency,Booking Date,Assigned Facility,Projected Release Date,Charge Type Counts
0,834075,"Adamson, Antawn Deshawn",24,Male,Black,6 ft 0 in,165 lbs,Black,Brown,Portland Sheriff,10/04/2022 09:41 AM,MCDC,Unknown,{'PAROLE OR PPSV VIOL (U Felony)': 1}
1,720433,"Dobbs, Timothy John",40,Male,Black,5 ft 9 in,160 lbs,Black,Brown,"Portland Police, Other",02/28/2022 06:55 AM,MCDC,Unknown,"{'FELON POSS FA (C Felony)': 4, 'ATT MURDER II..."
2,810789,"Porter, Greg Phillip",65,Male,White,5 ft 6 in,240 lbs,White,Blue,"Portland Police, Central Precinct",05/25/2018 09:42 PM,MCIJ,Unknown,"{'ATT AGGR MURDER (A Felony)': 6, 'ASSAULT I (..."
3,761726,"Mccool, Joel Matthew",31,Male,Black,5 ft 7 in,160 lbs,Black,Brown,MCSO Transports,07/05/2018 02:29 PM,MCIJ,Unknown,"{'UNLAW USE WEAPON (C Felony)': 19, 'ATT AGGR ..."
4,707888,"Hartrasberry, Jamias Leondre",34,Male,Black,5 ft 7 in,160 lbs,Black,Brown,"Portland Police, Other",09/01/2018 09:55 AM,MCIJ,Unknown,"{'MURDER I (A Felony)': 2, 'DEL COCAINE (B Fel..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,444581,"Campbell, James Dutton",53,Male,White,5 ft 11 in,202 lbs,Brown,Blue,Tri-Met Police,10/04/2022 05:20 PM,MCDC,Unknown,{'COUNTY HOLD (U Unknown)': 1}
852,773340,"Cabrera, Louis Jonathan",29,Male,Hispanic,5 ft 5 in,230 lbs,Black,Brown,Portland Sheriff,10/04/2022 02:32 PM,MCDC,Unknown,"{'UNLAW USE WEAPON (C Felony)': 2, 'MENACING (..."
853,818010,"Brasuell, Joseph David",23,Male,White,5 ft 10 in,140 lbs,Brown,Blue,MCSO Transports,10/04/2022 11:58 AM,MCDC,Unknown,{'POSS ILLEG WEAPON (B Felony)': 1}
854,757355,"Baker, Kevin Wayne",57,Male,White,6 ft 1 in,200 lbs,Black,Brown,NaN,10/04/2022 07:41 AM,MCDC,10/08/2022,{'ROBBERY III (C Felony)': 1}


In [30]:
inmates_details_df.dropna()

,SWIS ID,Name,Age,Gender,Race,Height,Weight,Hair,Eyes,Arresting Agency,Booking Date,Assigned Facility,Projected Release Date,Charge Type Counts
0,831307,"Adam, Abdiweli",28,Male,Black,5 ft 7 in,130 lbs,Black,Brown,"Portland Police, Central Precinct",09/26/2022 05:54 PM,MCDC,Unknown,{'BURGLARY II (C Felony)': 1}
1,816992,"Birch, Jonathan Alexander",31,Male,White,5 ft 9 in,235 lbs,Brown,Brown,MCSO Transports,07/26/2022 12:31 PM,MCIJ,Unknown,"{'COUNTY HOLD (U Unknown)': 9, 'RECK ENDANGER ..."
2,810789,"Porter, Greg Phillip",65,Male,White,5 ft 6 in,240 lbs,White,Blue,"Portland Police, Central Precinct",05/25/2018 09:42 PM,MCIJ,Unknown,"{'ATT AGGR MURDER (A Felony)': 6, 'ASSAULT I (..."
3,761726,"Mccool, Joel Matthew",31,Male,Black,5 ft 7 in,160 lbs,Black,Brown,MCSO Transports,07/05/2018 02:29 PM,MCIJ,Unknown,"{'UNLAW USE WEAPON (C Felony)': 19, 'ATT AGGR ..."
4,86303,"Garner, Brandon Paul",54,Male,Black,6 ft 1 in,175 lbs,Black,Brown,"Portland Police, East Precinct",08/31/2018 10:37 PM,MCIJ,Unknown,"{'PROM PROST (C Felony)': 3, 'COMPELL PROST (B..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,795262,"Cockrell, Jocelyn D",40,Female,White,5 ft 5 in,170 lbs,Brown,Brown,MCSO Transports,09/26/2022 11:57 AM,MCDC,Unknown,{'TRESPASS I (A Misdemeanor)': 1}
868,811647,"Burke, Sean Michael",36,Male,White,5 ft 11 in,170 lbs,Brown,Brown,"Portland Police, East Precinct",09/26/2022 12:18 PM,MCDC,Unknown,"{'COUNTY HOLD (U Unknown)': 1, 'FUGITIVE (U Fe..."
869,743033,"Broughton, James",32,Male,Black,5 ft 8 in,143 lbs,Black,Brown,Community Corrections,09/26/2022 03:47 PM,MCDC,Unknown,{'PAROLE OR PPSV VIOL (U Felony)': 1}
870,672315,"Alatorre, Trinidad",38,Male,White,5 ft 9 in,190 lbs,Brown,Brown,MCSO Transports,09/26/2022 11:46 AM,MCIJ,Unknown,"{'UNLAW USE WEAPON (C Felony)': 1, 'MENACING (..."


In [28]:
gender_counts = {**gender_counts, 'Date': currentDate}

In [22]:
pd.DataFrame([gender_counts]).to_csv('csvs/inmate_counts_gender.csv', mode='a',  index=False, header=False)

In [17]:
inmates_details_df['Race'].value_counts()

White                         428
Black                         297
Hispanic                       99
Asian                          27
Native American or Alaskan     14
P                               7
Name: Race, dtype: int64

In [8]:
inmates_details_df['Hair'].value_counts()

Black     400
Brown     361
Blonde     51
Gray       23
XXX        21
Red         8
White       1
Name: Hair, dtype: int64

In [9]:
inmates_details_df['Eyes'].value_counts()

Brown    616
Blue     147
Hazel     55
Green     39
Black      5
Pink       1
XXX        1
Gray       1
Name: Eyes, dtype: int64

In [14]:
inmates_details_df['Arresting Agency'].value_counts()

Portland Police, East Precinct       167
Portland Police, Central Precinct    136
MCSO Transports                      118
Portland Police, North Precinct       87
Portland Police, Other                82
US Marshals Service                   77
Gresham Police Department             54
Portland Sheriff                      51
Community Corrections                 34
Tri-Met Police                        17
Portland Criminal Court                9
Multnomah County Sheriff Booking       9
Other Agency                           7
Port of Portland                       6
Drug Enforcement Administration        5
OHSU University Police                 2
PSU                                    1
Name: Arresting Agency, dtype: int64

In [15]:
inmates_details_df['Assigned Facility'].value_counts()

MCIJ    520
MCDC    353
Name: Assigned Facility, dtype: int64

In [16]:
inmates_details_df['Projected Release Date'].value_counts()

Unknown       742
09/18/2022      7
09/20/2022      5
09/19/2022      5
10/09/2022      5
             ... 
11/11/2022      1
10/21/2022      1
10/29/2022      1
01/03/2023      1
01/13/2023      1
Name: Projected Release Date, Length: 75, dtype: int64

In [10]:
inmates_details_df['Age'].median()

35.0